# Introduction to SAR

## High level overview
Synthetic aperature radar (SAR) is a form of pulsed doppler radar where a large aperture is synthesized because the equivalent physical aperture is not physically realizable.  SAR has three main collection modes:
1. Spotlight (Antenna is trained on a single point during the collection)
    - This is sometimes referred to as frame
2. Stripmap (Antenna is trained on a fixed line during the collection and the antenna is fixed)
3. Scan (Antenna is trained on a fixed line during the collection and the antenna is changing where its pointing)
    - Stripmap is a special case of scan mode when the antenna is not changing pointing direction during the collection

![sar_modes](./images/sar_modes.png)  

Each mode is used for different purposes.  Spotlight mode is usually used to collect smaller area higher resolution images whereas scan/stripmap are used to collect larger areas.

# Geometry
SAR is very geometry heavy!  Before we talk about SAR it's good to provide some general knowledge on geometry/coordinates.

## Coordinate conversion

### Vectors
To represent coordinates we use vectors: https://www.youtube.com/watch?v=fNk_zzaMoSs.  Vectors are mathematical objects which describe direction and magnitude.  An example of a vector in $\mathbb{R}^2$ is $v = [1, 0]$

### Coordinate system
We define a coordinate system by two properties:
1. basis: https://en.wikipedia.org/wiki/Basis_(linear_algebra)
2. vector origin
For a refresher on this see: https://www.youtube.com/watch?v=k7RM-ot2NWY

### Changing coordinate systems
The act of converting a vector defined in one coordinate system to another is known as a change of basis.  For more details see (https://www.youtube.com/watch?v=kYB8IZa5AuE)


## Common terms
The following are some common terms related to SAR that will come up frequently:
* Aperture reference point (ARP): Position of the radar (antenna)
  - This is sometimes referred to as aperture phase center (APC)
  - Often represented as a polynomial
* Scene center point (SCP): Position of the center of the image
  - Some people will refer to this as the stabilization reference point (SRP)
* Ground reference point (GRP): position of antenna aimpoint
  - Often represented as a polynomial
* Center of aperture (COA): Position of the sensr at the center of the collection
* Dwell time: Total time spent collecting data
* Dwell angle: Angle between the line of sight vector at the beginning of the collection and the line of sight vector at the end of the collection
* Line of sight (LOS): pointing vector from the ARP to the SCP
* Grazing angle: angle between the LOS and the ground plane
* Slant range: Distance from the ARP to the SCP
  - magnitude of the line of sight 
* Range swath: Distance of the min/max range collected
  - Sometimes represented as time or distance
* Tilt angle: Angle between the ground plane z basis vector and the slant plane z basis vector
* Squint angle: Angle between the line of sight vector and the ARP velocity vector

![sar_geom](./images/sar_geom.png)  

## Earth-centered, Earth-fixed (ECEF)
The ECEF coordinate system is most commonly used coordinate system when dealing with coordinates in SAR: https://en.wikipedia.org/wiki/Earth-centered,_Earth-fixed_coordinate_system

Often times we will convert from local coordinate frames to our problem (slant plane, ground plane, east north up (ENU), ...) to a global coordinate frame like ECEF better represent coordites for modeling.  
To convert from a local coordinate frame to a ECEF compute the following:
$$\mathbf{p}_{\text{local}} = \mathbf{R}^{T}_{\text{local}\rightarrow\text{ecef}}(\mathbf{p}_{ecef} - \mathbf{p}_{\text{origin}})$$
where:
* $\mathbf{R}_{\text{local}\rightarrow\text{ecef}}$ is an orthonormal rotation matrix which defines the coordinate system (its basis)
* $\mathbf{p}_{ecef}$ is the point in ECEF to convert to the local coordinate system
* $\mathbf{p}_{\text{origin}}$ is the origin of the local ooridnate system in ECEF

To convert from ECEF to a local frame compute the following:
$$\mathbf{p}_{\text{ecef}} = \mathbf{R}_{\text{local}\rightarrow\text{ecef}}\mathbf{p}_{local} + \mathbf{p}_{\text{origin}}$$


## Slant plane
The slant plane is the imaging plane directly along the line of sight of the radar to the scene:  

![sar_modes](./images/sar_planes.svg)  

The definition of the slant plane coordinate system is:
$$
\text{look} = \begin{cases}
1 & (\mathbf{u}_{p_{coa}} \times \mathbf{u}_{v_{coa}}) \cdot \mathbf{u}_{los},\\
0  & 0 \text{ else}
\end{cases}
$$  

$$\mathbf{z}_{sp} = \text{look}\cdot (\mathbf{u}_{v_{coa}} \times \mathbf{u}_{los})$$
$$\mathbf{x}_{sp} = -\mathbf{u}_{los}$$
$$\mathbf{y}_{sp} = \mathbf{z}_{sp} \times \mathbf{x}_{sp}$$
$$\mathbf{R}_{\text{sp}\rightarrow\text{ecef}} = [\mathbf{x}_{sp}, \mathbf{y}_{sp}, \mathbf{z}_{sp}]$$
To convert an ecef position to slant plane position compute:
$$\mathbf{p}_{\text{sp}} = \mathbf{R}_{\text{sp}\rightarrow\text{ecef}}(\mathbf{p}_{\text{ecef}} - \mathbf{p}_{\text{scp}})$$


## Ground plane
The ground plane is the plane that is tangent to the earth surface at the scene center point which can be seen in the picture in the slant plane section.
$$\mathbf{z}_{gp} = \frac{\mathbf{p}_{\text{scp}}}{|\mathbf{p}_{\text{scp}}|}$$
$$\mathbf{y}_{gp} = \frac{\mathbf{z}_{gp}\times(\mathbf{p}_{\text{arp}} - \mathbf{p}_{\text{scp}})}{|\mathbf{z}_{gp}\times(\mathbf{p}_{\text{arp}} - \mathbf{p}_{\text{scp}})|}$$
$$\mathbf{x}_{gp} = \mathbf{y}_{gp} \times \mathbf{z}_{gp}$$
$$\mathbf{R}_{\text{gp}\rightarrow\text{ecef}} = [\mathbf{gp}_x, \mathbf{gp}_y, \mathbf{gp}_z]$$

To convert an ecef position to a ground plane position compute:
$$\mathbf{p}_{\text{gp}} = \mathbf{R}_{\text{gp}\rightarrow\text{ecef}}(\mathbf{p}_{\text{ecef}} - \mathbf{p}_{\text{scp}})$$



## Ground to slant and slant to ground conversion
More complicated! We'll examine this problem more when we talk about converting a slant plane image to a ground plane image (sicd image to detected image chain)

# Aperture synthesis
The problem with real aperture radar is that to achieve fine azimuth beamwidth (azimuth resolution) we require a phsyically large antenna.  The beamwidth of an antenna is given by:
$$\beta = \frac{\lambda}{D}$$
Where $\lambda$ is the wavelength and $D$ is the physical aperture size.  The resolution (width of the antenna on the ground) is given by:
$$\rho_{a} = R\beta = \frac{R\lambda}{D}$$
where $R$ is the slant range from the sensor to the ground.  Suppose we wanted a resolution of $\rho_a = 1 \text{ meters}$ with a wavelength of $\lambda = 0.03 \text{ meters}$ at a slant range of $R = 50 \text { km}$.  Solving for $D$ and plugging these numbers in yields:
$$D = \frac{\lambda R}{\rho_a} = \frac{0.03 \cdot 50\cdot 10^{3}}{1} = 1500 \text{ meters}$$  

This antenna size is physically unrealizable to fabricate and mount on a flying platform.  The solution is to use a wide beamwidth antenna and synthesize an antenna array to achieve a much narrower beamwidth.  

## Ambiguities
Ambiguities in SAR are points on the earth that have the same signal characteristics as a point in the illuminated scene.  This means that the signal characteristics are indistinguishable which means the point outside the scene appears to be identical to the point inside the scene.  This is undesirable because we want to be able to claim that each point in our image is unique.  

Ambiguities have two forms in SAR:
* Range ambiguitiies
* Doppler ambiguities

Ambiguites will look like replicas of objects at different ranges/azimuth positions, but that object is not real; it is merely a copy.

### Range Ambiguities
Range ambiguities are purely a factor of geometry.  The ambiguious range relative to the unambiguous range to a point inside the scene $R_0$ is defined as:
$$R_{k} = R_0 + k \frac{c}{2 PRF}, \quad k \in \cdots -2, -1, 0, 1, 2, \cdots$$  

This means that for a point a distance of 1000m away from the sensor with a PRF of 1000 Hz, that:
|$R_k$| distance (m)|
| --- | ----------- |
| $R_{-2}$ | -299 km |
| $R_{-1}$ | -149 km |
| $R_{0}$ | 0 km (this is the un-ambiguous distance to the point) |
| $R_{1}$ | 151 km |
| $R_{2}$ | 301 km |

Range ambiguities are driven by the collection geometry.  They occur when the pulse continues to travel hits a point outside the receive window, then shows up in the return of a pulse later in the collection.  Since the pulse has to travel farther and received power falls off with $\frac{1}{R}$, range ambiguties power is often below the noise.  It is the tasking software's goal to figure out range ambiguity power for the collection and to task the system to try to minimize the power of the ambiguities. 


### Doppler ambiguities
Along each iso-range line doppler ambiguities exist at integer multiples of the doppler bandwidth (PRF) of the scene:
$$f_{m} = f_0 + m B, \quad m \in \cdots -2, -1, 0, 1, 2, \cdots $$
Since doppler frequency is only along the projection to the line of sight vector you will often see this equation re-written as:
$$2\frac{\mathbf{u}_{los} \cdot \mathbf{v}_{ARP}}{c} = 2\frac{\mathbf{u}_{los} \cdot \mathbf{v}_{ARP}}{c} + m PRF \quad m \in \cdots -2, -1, 0, 1, 2, \cdots $$  

Since the scene extent is controlled by how fast we sample the scene in Doppler (azimuth), doppler ambiguities are the same concept of aliasing we described in the sampling problem in the math/dsp intro.  Doppler ambiguities are usually attenuated by the antenna beam however strong reflecting objects in the sidelobes or moving targets can alias back into the scene and show up as ambiguities.  These are sometimes referred to as "fuzz balls" since when they alias in they are so out of focus relative to the motion compensation point

### Visualization

![ambiguities](./images/ambiguities.PNG)

### Tradeoffs
When designing a system the following relationship between range/doppler ambiguities should be noted:
|PRF | Range Ambiguities| Doppler Ambiguities|
| -- | ---------------- | ------------------ |
| increases | move in | move out | 
| decreases | move out | move in |

Techniques such as multiple azimuth beam (MAB) sample the scene sub-nyquist (very low PRF) so that the range ambiguities are so far away that the range swath can be massive, then use the multiple antenna beams to cancel out the increased doppler ambiguities  

## File formats associated with SAR products
There are three main file formats users will interact with when working with SAR data.  These file formats were all developed by NGA to standardize SAR products in the community:
* Compensated phase history data (CPHD) https://nsgreg.nga.mil/doc/view?i=4638
    - NOT AN IMAGE
    - SAR phase history data that has been motion compensated and matched filter
    - Sensor specific hardware corrections have also been applied 
* Sensor independent complex data (SICD) https://nsgreg.nga.mil/doc/view?i=4900
    - Image in the slant plane
    - pixels are complex float
    - mainly used for exploitation purposes (automatic target recognition, and paired product generation) since phase information is present
* Sensor independent derived data (SIDD) https://nsgreg.nga.mil/doc/view?i=5009
    - Image in the ground plane
    - Pixels are 8 bit unsigned integers (values range from 0 - 255)
    - This is what most SAR image consumers view

Coming soon: Compensated receive signal data (CRSD)!  CRSD has not been matched filtered and mocomp'd only for the receive portion.